# Tutorial 01: LCEL Básico - Tradutor Inteligente

**Objetivos de Aprendizagem:**
- Dominar LCEL (LangChain Expression Language) para criar chains elegantes
- Usar ChatPromptTemplate para prompts conversacionais
- Implementar Output Parsers para formatação de respostas
- Construir sistemas de tradução multi-idioma
- Validar idiomas e detectar erros automaticamente

**Pré-requisitos:**
- Tutorial 01-LLMs-Basicos: Introdução aos LLMs concluído
- Tutorial 02-Prompts: ChatPromptTemplate (recomendado)
- Compreensão básica de Python e APIs

---

## Introdução

Este tutorial demonstra como usar LCEL (LangChain Expression Language) para criar chains elegantes e funcionais. Vamos construir um tradutor inteligente que serve como exemplo prático dos conceitos fundamentais do LangChain.

### O que é LCEL?

**LCEL (LangChain Expression Language)** é a sintaxe moderna do LangChain que usa o operador `|` (pipe) para conectar componentes de forma elegante e legível.

**Vantagens:**
- Código mais limpo e legível
- Flexibilidade para combinações complexas
- Substitui a sintaxe antiga de chains
- Facilita manutenção e entendimento


### Conceitos-Chave

**LCEL (LangChain Expression Language):**
- Sintaxe moderna usando operador `|` para conectar componentes
- Código mais limpo e legível
- Flexibilidade para combinações complexas
- Facilita manutenção

**ChatPromptTemplate:**
- Para modelos conversacionais (ChatGPT, etc.)
- Inclui roles (system, human, assistant)
- Melhor controle sobre contexto da conversa
- Mais estruturado que PromptTemplate simples

**Output Parsers:**
- StrOutputParser: Converte resposta em string simples
- PydanticOutputParser: Converte em objeto estruturado
- JSONOutputParser: Converte em JSON válido
- Custom Parsers: Parsers personalizados

**Validação:**
- Validação de idiomas suportados
- Detecção de erros na tradução
- Tratamento de exceções robusto

### Casos de Uso

**Aplicações Reais:**
- Sistema de tradução para aplicações web
- Chatbot multi-idioma para atendimento
- Plataforma de e-learning com suporte internacional
- Sistema de localização para produtos digitais

**Habilidades Desenvolvidas:**
- Fundamentos do LangChain com LCEL
- Criação de prompts eficazes
- Validação de dados com Pydantic
- Tratamento de erros robusto
- Arquitetura modular e reutilizável

In [ ]:
# Configuração inicial
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Verificar se OPENAI_API_KEY está configurada
if not os.getenv('OPENAI_API_KEY'):
    print("⚠️ ATENÇÃO: OPENAI_API_KEY não encontrada no .env")
    print("   Configure a chave antes de continuar.")
else:
    print("✅ Variáveis de ambiente carregadas com sucesso!")

Chave da API carregada com sucesso!


## Exemplo 1: Criar Template de Prompt

Vamos criar um template de prompt para tradução usando ChatPromptTemplate.

**Por que ChatPromptTemplate?**
- Mais estruturado que PromptTemplate simples
- Permite definir roles (system, human, assistant)
- Melhor controle sobre contexto da conversa
- Ideal para modelos de chat (ChatOpenAI)

In [ ]:
# Criar template de prompt para tradução
prompt_template = ChatPromptTemplate.from_messages([
    # Mensagem do sistema define o comportamento da IA
    ("system", "Você é um tradutor especialista. Sua resposta deve conter apenas o texto traduzido, sem nenhuma explicação adicional."),
    # Mensagem do usuário com variáveis que serão preenchidas
    ("human", "Traduza o seguinte texto para o idioma '{idioma}':\n\n{texto}")
])

print("Template criado com sucesso!")
print(f"Variáveis esperadas: {prompt_template.input_variables}")
print(f"Número de mensagens: {len(prompt_template.messages)}")

Variáveis esperadas: ['idioma', 'texto']
Número de mensagens: 2


## Exemplo 2: Configurar o Modelo

Vamos configurar o modelo ChatOpenAI para traduções precisas.

**Por que temperature=0.0?**
- Traduções precisas não precisam de criatividade
- 0.0 = mais determinístico (mesma resposta para mesmo input)
- 1.0 = mais criativo (respostas variadas)

**ChatOpenAI vs OpenAI:**
- ChatOpenAI: Interface estruturada com mensagens (recomendado)
- OpenAI: Interface simples de texto (mais básico)

In [ ]:
# Configurar modelo de chat
chat_model = ChatOpenAI(
    temperature=0.0,  # Traduções precisas não precisam de criatividade
    model="gpt-3.5-turbo"
)

print(f"Modelo configurado: {chat_model.model_name}")
print(f"Temperatura: {chat_model.temperature}")

Modelo: gpt-3.5-turbo
Temperatura: 0.0


## Exemplo 3: Construir Chain com LCEL

Vamos construir uma chain usando LCEL (LangChain Expression Language).

**O que é LCEL?**
- Sintaxe moderna do LangChain
- Usa operador `|` (pipe) para conectar componentes
- Código mais limpo e legível
- Facilita manutenção

**Como funciona:**
```python
prompt | model | parser
```

Cada `|` passa a saída de um componente para a entrada do próximo:
1. **prompt** → formata o texto com as variáveis
2. **model** → gera a resposta
3. **parser** → processa a saída final

É a sintaxe moderna do LangChain que usa o operador `|` (pipe) para conectar componentes de forma elegante.

### **Como funciona o operador `|`:**

```python
prompt | model | parser
```

Cada `|` passa a saída de um componente para a entrada do próximo:
1. **prompt** → formata o texto com as variáveis
2. **model** → gera a resposta
3. **parser** → processa a saída final

In [25]:
# 5. CONSTRUINDO A CHAIN COM LCEL

# O parser de saída para garantir que o resultado seja uma string
output_parser = StrOutputParser()

# A sintaxe de pipe '|' encadeia os componentes em um fluxo lógico
tradutor_chain = prompt_template | chat_model | output_parser

print(f"Tipo da chain: {type(tradutor_chain)}")
print(f"Componentes: prompt → model → parser")

Tipo da chain: <class 'langchain_core.runnables.base.RunnableSequence'>
Componentes: prompt → model → parser


In [26]:
# 6. PRIMEIRO TESTE - TRADUÇÃO SIMPLES

# Texto para traduzir
texto_para_traduzir = "LangChain is a powerful framework for developing applications powered by language models."
idioma_alvo = "Português do Brasil"

print("🔄 Executando tradução...")
print(f"Texto original: {texto_para_traduzir}")
print(f"Idioma alvo: {idioma_alvo}")
print("-" * 60)

# A entrada da chain é um dicionário com chaves que correspondem às variáveis do prompt
response = tradutor_chain.invoke({
    "idioma": idioma_alvo,
    "texto": texto_para_traduzir
})

print(f"Resultado: {response}")

🔄 Executando tradução...
Texto original: LangChain is a powerful framework for developing applications powered by language models.
Idioma alvo: Português do Brasil
------------------------------------------------------------
Resultado: LangChain é um framework poderoso para desenvolver aplicativos alimentados por modelos de linguagem.


In [27]:
# 7. TESTE COM MÚLTIPLOS IDIOMAS

texto_teste = "Artificial Intelligence is transforming the world."
idiomas_teste = ["Espanhol", "Francês", "Alemão", "Italiano", "Japonês"]

print("🌍 Testando tradução para múltiplos idiomas...")
print(f"Texto original: {texto_teste}")
print("=" * 60)

for idioma in idiomas_teste:
    print(f"\n🔄 Traduzindo para {idioma}...")
    
    try:
        traducao = tradutor_chain.invoke({
            "idioma": idioma,
            "texto": texto_teste
        })
        print(f"✅ {idioma}: {traducao}")
    except Exception as e:
        print(f"❌ Erro ao traduzir para {idioma}: {e}")

print("\n�� Teste de múltiplos idiomas concluído!")

🌍 Testando tradução para múltiplos idiomas...
Texto original: Artificial Intelligence is transforming the world.

🔄 Traduzindo para Espanhol...
✅ Espanhol: La inteligencia artificial está transformando el mundo.

🔄 Traduzindo para Francês...
✅ Francês: L'intelligence artificielle transforme le monde.

🔄 Traduzindo para Alemão...
✅ Alemão: Künstliche Intelligenz verändert die Welt.

🔄 Traduzindo para Italiano...
✅ Italiano: L'intelligenza artificiale sta trasformando il mondo.

🔄 Traduzindo para Japonês...
✅ Japonês: 人工知能は世界を変えています。

�� Teste de múltiplos idiomas concluído!


## 🎯 Desafios para Praticar

### **Desafio 1: Tradução Interativa**
Modifique o código para que ele peça ao usuário para digitar o texto e o idioma no terminal.

**Dica:** Use `input()` para capturar entrada do usuário.


In [31]:
def traduzir_interativo():
    print("�� TRADUTOR INTERATIVO")
    print("=" * 40)
    
    # Captura entrada do usuário
    texto = input("Digite o texto para traduzir: ")
    idioma = input("Digite o idioma de destino: ")
    
    print(f"\n�� Traduzindo '{texto}' para {idioma}...")
    
    try:
        traducao = tradutor_chain.invoke({
            "idioma": idioma,
            "texto": texto
        })
        print(f"✅ Tradução: {traducao}")
    except Exception as e:
        print(f"❌ Erro na tradução: {e}")

traduzir_interativo()

�� TRADUTOR INTERATIVO

�� Traduzindo '人工知能は世界を変えています' para portugues...
✅ Tradução: A inteligência artificial está mudando o mundo.



### **Desafio 2: Tradução Reversa**
Crie uma função que traduza um texto para um idioma e depois traduza de volta para o idioma original.


In [29]:
def traducao_reversa(texto_original, idioma_intermediario):
    """
    Traduz um texto para um idioma intermediário e depois de volta para o original
    """
    print(f"🔄 DESAFIO 2: Tradução Reversa")
    print(f"Texto original: {texto_original}")
    print(f"Idioma intermediário: {idioma_intermediario}")
    print("-" * 50)
    
    try:
        # Primeira tradução: original → intermediário
        print("1️⃣ Traduzindo para idioma intermediário...")
        traducao_intermediaria = tradutor_chain.invoke({
            "idioma": idioma_intermediario,
            "texto": texto_original
        })
        print(f"✅ Tradução intermediária: {traducao_intermediaria}")
        
        # Segunda tradução: intermediário → original
        print("\n2️⃣ Traduzindo de volta para o idioma original...")
        traducao_final = tradutor_chain.invoke({
            "idioma": "Português",  # Assumindo que o original é português
            "texto": traducao_intermediaria
        })
        print(f"✅ Tradução final: {traducao_final}")
        
        # Comparação
        print("\n📊 COMPARAÇÃO:")
        print(f"Original:     {texto_original}")
        print(f"Final:        {traducao_final}")
        
        # Verifica se são similares
        if texto_original.lower() == traducao_final.lower():
            print("🎯 Resultado: Traduções idênticas!")
        else:
            print("🤔 Resultado: Traduções similares mas não idênticas")
            
    except Exception as e:
        print(f"❌ Erro na tradução reversa: {e}")

# Teste do desafio 2
texto_teste = "Olá, como você está hoje?"
traducao_reversa(texto_teste, "Inglês")

🔄 DESAFIO 2: Tradução Reversa
Texto original: Olá, como você está hoje?
Idioma intermediário: Inglês
--------------------------------------------------
1️⃣ Traduzindo para idioma intermediário...
✅ Tradução intermediária: Hello, how are you today?

2️⃣ Traduzindo de volta para o idioma original...
✅ Tradução final: Olá, como você está hoje?

📊 COMPARAÇÃO:
Original:     Olá, como você está hoje?
Final:        Olá, como você está hoje?
🎯 Resultado: Traduções idênticas!



### **Desafio 3: Validação de Idiomas**
Crie uma lista de idiomas suportados e valide se o idioma digitado pelo usuário está na lista.

In [30]:
# Lista de idiomas suportados
IDIOMAS_SUPORTADOS = {
    "português": ["português", "portugues", "pt", "pt-br", "português do brasil"],
    "inglês": ["inglês", "ingles", "en", "english"],
    "espanhol": ["espanhol", "es", "español", "castelhano"],
    "francês": ["francês", "frances", "fr", "français"],
    "alemão": ["alemão", "alemao", "de", "deutsch"],
    "italiano": ["italiano", "it", "italian"],
    "japonês": ["japonês", "japones", "ja", "nihongo"],
    "chinês": ["chinês", "chines", "zh", "mandarim"],
    "coreano": ["coreano", "ko", "hangugeo"],
    "russo": ["russo", "ru", "russkiy"]
}

def validar_idioma(idioma_digitado):
    """
    Valida se o idioma digitado está na lista de suportados
    Retorna o idioma padronizado ou None se não for válido
    """
    idioma_limpo = idioma_digitado.lower().strip()
    
    for idioma_principal, variacoes in IDIOMAS_SUPORTADOS.items():
        if idioma_limpo in variacoes:
            return idioma_principal
    
    return None

def tradutor_com_validacao():
    """
    Tradutor que valida o idioma antes de traduzir
    """
    print("🔒 TRADUTOR COM VALIDAÇÃO DE IDIOMAS")
    print("=" * 50)
    
    # Mostra idiomas disponíveis
    print("🌍 Idiomas disponíveis:")
    for i, idioma in enumerate(IDIOMAS_SUPORTADOS.keys(), 1):
        print(f"   {i}. {idioma.title()}")
    
    print("\n" + "-" * 50)
    
    # Captura entrada do usuário
    texto = input("Digite o texto para traduzir: ")
    idioma_destino = input("Digite o idioma de destino: ")
    
    # Valida o idioma
    idioma_valido = validar_idioma(idioma_destino)
    
    if idioma_valido:
        print(f"\n✅ Idioma válido: {idioma_valido.title()}")
        print(f"�� Traduzindo '{texto}' para {idioma_valido.title()}...")
        
        try:
            traducao = tradutor_chain.invoke({
                "idioma": idioma_valido.title(),
                "texto": texto
            })
            print(f"✅ Tradução: {traducao}")
        except Exception as e:
            print(f"❌ Erro na tradução: {e}")
    else:
        print(f"\n❌ Idioma '{idioma_destino}' não é suportado!")
        print("💡 Dica: Use um dos idiomas da lista acima")
        
        # Sugere idiomas similares
        sugestoes = []
        for idioma_principal in IDIOMAS_SUPORTADOS.keys():
            if idioma_destino.lower() in idioma_principal or idioma_principal in idioma_destino.lower():
                sugestoes.append(idioma_principal)
        
        if sugestoes:
            print(f"🔍 Sugestões similares: {', '.join(sugestoes)}")

tradutor_com_validacao()

🔒 TRADUTOR COM VALIDAÇÃO DE IDIOMAS
🌍 Idiomas disponíveis:
   1. Português
   2. Inglês
   3. Espanhol
   4. Francês
   5. Alemão
   6. Italiano
   7. Japonês
   8. Chinês
   9. Coreano
   10. Russo

--------------------------------------------------

✅ Idioma válido: Português
�� Traduzindo '人工知能は世界を変えています' para Português...
✅ Tradução: A inteligência artificial está mudando o mundo.
